# Financial Derivatives Individual Project

## Francisco Perestrello | 20241560

In [597]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime

from scipy.stats import norm

### 1. Select a stock or index with liquid options in the 1 to 12 month range. Name your stock/Index.

In [633]:
# Define the ticker symbol for S&P 500 (^SPX)
ticker_symbol = "^SPX"
ticker = yf.Ticker(ticker_symbol)

# Fixed dates for reproducibility
yesterday = datetime.datetime(2025, 4, 22) 
today = datetime.datetime(2025, 4, 23)

# Get the most recent data for the underlying 
underlying_data = ticker.history(start=yesterday, end=today)
current_time = underlying_data.index[-1]
current_price = underlying_data['Close'].iloc[-1]

print(f"Underlying price at {current_time}: ${current_price:.2f}")

Underlying price at 2025-04-22 00:00:00-04:00: $5287.76


The S&P 500 options are among the most actively traded in the world, with high daily volume and open interest across a wide range of strike prices and expiration dates. This ensures minimal slippage and ease of execution for a box spread.
As a broad market index, the S&P 500 is less volatile than individual stocks, which suits our risk-averse profile, even though a box spread payoff is theoretically risk-free.

#### a) Pick an expiration timeframe within the next 12 weeks.

In [31]:
# List to store open interest data
oi_data = []

# Get available option expiration dates (as strings in 'YYYY-MM-DD')
expiration_dates = ticker.options

# Loop over each expiration date
for exp in expiration_dates:
    # Get the option chain for this expiration date
    chain = ticker.option_chain(exp)
    calls = chain.calls
    puts = chain.puts

    # Sum the open interest for calls and puts
    total_call_oi = calls['openInterest'].sum()
    total_put_oi = puts['openInterest'].sum()
    total_oi = total_call_oi + total_put_oi

    # Convert the expiration date string to a datetime object
    exp_date = datetime.datetime.strptime(exp, '%Y-%m-%d')
    #month_year = exp_date.strftime('%Y-%m')

    # Append a record to our data list.
    oi_data.append({
        'expiration': exp,
        'calls_oi': total_call_oi,
        'puts_oi': total_put_oi,
        'total_oi': total_oi
    })

# Create a DataFrame from our data
df = pd.DataFrame(oi_data)
print(df[:31])

    expiration  calls_oi    puts_oi   total_oi
0   2025-04-22   77478.0   141746.0   219224.0
1   2025-04-23   42914.0    96410.0   139324.0
2   2025-04-24   22089.0    56554.0    78643.0
3   2025-04-25   99221.0   276461.0   375682.0
4   2025-04-28   27839.0    63663.0    91502.0
5   2025-04-29   17693.0    25355.0    43048.0
6   2025-04-30  212548.0   318584.0   531132.0
7   2025-05-01   15809.0    34979.0    50788.0
8   2025-05-02   76744.0   299689.0   376433.0
9   2025-05-05   10278.0    26517.0    36795.0
10  2025-05-06    7524.0    12687.0    20211.0
11  2025-05-07    8668.0    14709.0    23377.0
12  2025-05-08    4440.0    13407.0    17847.0
13  2025-05-09   35931.0   186523.0   222454.0
14  2025-05-12    5309.0     7777.0    13086.0
15  2025-05-13    1671.0     2895.0     4566.0
16  2025-05-14   10231.0    13102.0    23333.0
17  2025-05-15   10104.0     3670.0    13774.0
18  2025-05-16  352758.0   692653.0  1045411.0
19  2025-05-19    3362.0     3090.0     6452.0
20  2025-05-2

A box spread requires highly liquid options to minimize transaction costs and ensure tight bid-ask spreads, so we will choose an expiracy date with high total open interest which indicates greater liquidity, reducing bid-ask spreads and transaction costs. We also want options that are sufficiently far-out from the present date to allow for meaningful analysis of the box spread’s risk-return profile.

For these reasons, the selected expiracy date will be 2025-06-20. Dated to expire in about 8 weeks, it has a total Open Interest of 2,551,933, balanced between calls and puts.

In [34]:
# Get selected expiration date
target_date = df['expiration'][28]
print(target_date)

2025-06-20


#### b) Select the options you would use to create a box spread.

In [80]:
# Retrieve the option chain for the target expiration date
option_chain = ticker.option_chain(target_date)
calls = option_chain.calls.copy()
puts = option_chain.puts.copy()

# Define the strike range: ± 5%
lower_bound = current_price * 0.95
upper_bound = current_price * 1.05

# Filter calls and puts for strikes within the range
filtered_calls = calls[calls['strike'].between(lower_bound, upper_bound)].copy()
filtered_puts = puts[puts['strike'].between(lower_bound, upper_bound)].copy()

print('Calls around current price:\n')
print(filtered_calls)
print('\nPuts around current price:\n')
print(filtered_puts)

Calls around current price:

          contractSymbol             lastTradeDate  strike  lastPrice    bid  \
125  SPXW250620C05025000 2025-04-21 17:31:49+00:00  5025.0     304.13  412.1   
126  SPXW250620C05030000 2025-04-22 13:34:34+00:00  5030.0     349.85  407.0   
127   SPX250620C05035000 2025-04-21 18:17:00+00:00  5035.0     291.40  406.6   
128   SPX250620C05040000 2025-04-11 18:08:52+00:00  5040.0     507.27  404.7   
129   SPX250620C05045000 2025-04-17 14:42:28+00:00  5045.0     391.51  398.6   
..                   ...                       ...     ...        ...    ...   
226   SPX250620C05530000 2025-04-22 15:03:05+00:00  5530.0      96.17  113.1   
227  SPXW250620C05535000 2025-04-22 16:08:35+00:00  5535.0     107.60  112.6   
228  SPXW250620C05540000 2025-04-22 16:00:37+00:00  5540.0     110.07  110.8   
229  SPXW250620C05545000 2025-04-22 15:16:45+00:00  5545.0      94.50  108.5   
230   SPX250620C05550000 2025-04-22 16:28:10+00:00  5550.0     107.70  105.3   

       ask

In [50]:
# Find common strike prices
common_strikes = set(filtered_calls['strike']).intersection(set(filtered_puts['strike']))
filtered_calls = filtered_calls[filtered_calls['strike'].isin(common_strikes)]
filtered_puts = filtered_puts[filtered_puts['strike'].isin(common_strikes)]

# Add a column to differentiate between call and put quotes
filtered_calls['optionType'] = 'call'
filtered_puts['optionType'] = 'put'

# Combine the calls and puts into a single DataFrame
combined_df = pd.concat([filtered_calls, filtered_puts], ignore_index=True)

# Select columns of interest
columns_of_interest = ['optionType', 'strike', 'lastPrice', 'bid', 'ask', 'openInterest', 'volume', 'impliedVolatility']
combined_df = combined_df[columns_of_interest]

print(combined_df)

    optionType  strike  lastPrice    bid    ask  openInterest  volume  \
0         call  5025.0     304.13  403.1  417.3           146     9.0   
1         call  5030.0     349.85  401.6  414.0             2     2.0   
2         call  5035.0     291.40  400.5  402.8          1970     3.0   
3         call  5040.0     507.27  397.2  399.6          4803    91.0   
4         call  5045.0     391.51  393.5  395.8          1467     1.0   
..         ...     ...        ...    ...    ...           ...     ...   
207        put  5530.0     305.60  315.0  316.5           100     6.0   
208        put  5535.0     332.60  316.4  318.5           252     3.0   
209        put  5540.0     437.11  318.8  320.8           999     3.0   
210        put  5545.0     244.64  322.4  323.6            91     2.0   
211        put  5550.0     441.39  322.5  331.6           399     1.0   

     impliedVolatility  
0             0.305831  
1             0.305721  
2             0.295228  
3             0.295251 

I will be selecting the options based on:

- **Liquidity:** High open interest and tight bid-ask spreads.
- **Strike Difference:** The difference between strike prices (*K2 - K1*) should be reasonable to balance potential arbitrage profit with transaction costs.
- **Proximity to Current Price:** K1 should be below the current underlying price ($5287.51), and K2 should be above.

Seeing the options have 5 dollar increments, let us choose K1 = 5200 dollars, and K2 = 5400 dollars. Let us check the open interest and bid-ask spreads for these options.

In [166]:
# Select strikes for box spread
K1 = 5200
K2 = 5400

# Filter options for the box spread
buy_call = combined_df[(combined_df['optionType'] == 'call') & (combined_df['strike'] == K1)]
sell_call = combined_df[(combined_df['optionType'] == 'call') & (combined_df['strike'] == K2)]
buy_put = combined_df[(combined_df['optionType'] == 'put') & (combined_df['strike'] == K2)]
sell_put = combined_df[(combined_df['optionType'] == 'put') & (combined_df['strike'] == K1)]

# Display selected options
print("Box Spread Options:")
print(f"\nBuy Call (K1 = ${K1}):")
print(buy_call)
print(f"\nSell Call (K2 = ${K2}):")
print(sell_call)
print(f"\nBuy Put (K2 = ${K2}):")
print(buy_put)
print(f"\nSell Put (K1 = ${K1}):")
print(sell_put)

Box Spread Options:

Buy Call (K1 = $5200):
   optionType  strike  lastPrice    bid    ask  openInterest  volume  \
35       call  5200.0      269.1  287.3  288.4           267   109.0   

    impliedVolatility  
35           0.275465  

Sell Call (K2 = $5400):
   optionType  strike  lastPrice    bid    ask  openInterest  volume  \
75       call  5400.0     151.53  169.4  171.3         39550   410.0   

    impliedVolatility  
75           0.250551  

Buy Put (K2 = $5400):
    optionType  strike  lastPrice    bid    ask  openInterest  volume  \
181        put  5400.0      249.1  244.0  245.8         54062   176.0   

     impliedVolatility  
181           0.222948  

Sell Put (K1 = $5200):
    optionType  strike  lastPrice    bid    ask  openInterest  volume  \
141        put  5200.0     164.18  162.2  163.7         35223   693.0   

     impliedVolatility  
141           0.248323  


Indeed, all four options generally have high open interest and a tight bid-ask spread (between 1 and 2 dollars)

The call at $5200 has a moderate/low OI (267), but the tight spread (1.1) and decent volume (109) suggest it’s tradable.

#### c) Price the options with Black Scholes Merton approach to pricing

In [765]:
# Create a function to calculate BSM parameters
def bsm_parameters(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2

# Create a function to calculate BSM option price for a call option
def bsm_call_price(S0, K, T, r, sigma):
    d1, d2 = bsm_parameters(S0, K, T, r, sigma)
    C = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return C

# Create a function to calculate BSM option price for a call option
def bsm_put_price(S0, K, T, r, sigma):
    d1, d2 = bsm_parameters(S0, K, T, r, sigma)
    P = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    return P

# Create a function to return the risk-free rate
def get_risk_free_rate(period='3mo'):
    # Fetching the treasury rates data from Yahoo Finance
    treasury = yf.Ticker("^IRX")  # ^IRX is the symbol for 13-week (3-month) T-Bill (smallest existing timeframe in yahoo finance)
    hist = treasury.history(start=yesterday, end=today)  # Get data for the last year

    # Get the latest rate (current)
    latest_rate = hist['Close'].iloc[-1]

    # Convert the rate to a proportion (Yahoo Finance gives it in percentage points)
    risk_free_rate = latest_rate / 100
    return risk_free_rate

# Create a function to calculate time to expiration
def time_to_expiration(expiration_date_str):
    # Get today's date and the expiration date as date objects
    current_date = datetime.datetime(2025, 4, 22).date()
    exp_date = datetime.datetime.strptime(expiration_date_str, '%Y-%m-%d').date()
    
    # Count the number of trading days between the two dates
    trading_days = np.busday_count(current_date, exp_date)
    
    # Convert trading days to years (assuming 252 trading days per year)
    T = max(trading_days / 252, 0.0001)
    return T

In [419]:
# Set up BSM parameters
S0 = current_price
T = time_to_expiration(target_date)
r = get_risk_free_rate()

sigma_bc = buy_call['impliedVolatility'].values[0]
sigma_sc = sell_call['impliedVolatility'].values[0]
sigma_bp = buy_put['impliedVolatility'].values[0]
sigma_sp = sell_put['impliedVolatility'].values[0]

print(f"Stock Price: ${S0:.2f}")
print(f"Time to Maturity: {T:.4f}")
print(f"Risk Free Rate: {r:.2%}")
print(f"Buy Call Implied Volatility: {sigma_bc:.2%}")
print(f"Sell Call Implied Volatility: {sigma_sc:.2%}")
print(f"Buy Put Implied Volatility: {sigma_bp:.2%}")
print(f"Sell Put Implied Volatility: {sigma_sp:.2%}")

Stock Price: $5287.51
Time to Maturity: 0.1706
Risk Free Rate: 4.21%
Buy Call Implied Volatility: 27.55%
Sell Call Implied Volatility: 25.06%
Buy Put Implied Volatility: 22.29%
Sell Put Implied Volatility: 24.83%


In [421]:
# Calculate option prices
bsm_buy_call_price = bsm_call_price(S0, K1, T, r, sigma_bc)
bsm_sell_call_price = bsm_call_price(S0, K2, T, r, sigma_sc)
bsm_buy_put_price = bsm_put_price(S0, K2, T, r, sigma_bp)
bsm_sell_put_price = bsm_put_price(S0, K1, T, r, sigma_sp)

print(f"BSM Buy Call (K=${K1}) Option Price: ${bsm_buy_call_price:.2f}")
print(f"BSM Sell Call (K=${K2}) Option Price: ${bsm_sell_call_price:.2f}")
print(f"BSM Buy Put (K=${K2}) Option Price: ${bsm_buy_put_price:.2f}")
print(f"BSM Sell Put (K=${K1}) Option Price: ${bsm_sell_put_price:.2f}")

BSM Buy Call (K=$5200) Option Price: $304.64
BSM Sell Call (K=$5400) Option Price: $184.81
BSM Buy Put (K=$5400) Option Price: $234.67
BSM Sell Put (K=$5200) Option Price: $157.11


#### d) Price the options with real quote data, make sure you use the correct data assuming that you are actually going to construct this strategy.

We buy options at the 'ask' price and sell options at the 'bid' price, reflecting the real cost of entering the positions.

In [425]:
# Get real quote data option prices
actual_buy_call_price = buy_call['ask'].values[0]
actual_sell_call_price = sell_call['bid'].values[0]
actual_buy_put_price = buy_put['ask'].values[0]
actual_sell_put_price = sell_put['bid'].values[0]

print(f"Actual Buy Call (K=${K1}) Option Price: ${actual_buy_call_price:.2f}")
print(f"Actual Sell Call (K=${K2}) Option Price: ${actual_sell_call_price:.2f}")
print(f"Actual Buy Put (K=${K2}) Option Price: ${actual_buy_put_price:.2f}")
print(f"Actual Sell Put (K=${K1}) Option Price: ${actual_sell_put_price:.2f}")

Actual Buy Call (K=$5200) Option Price: $288.40
Actual Sell Call (K=$5400) Option Price: $169.40
Actual Buy Put (K=$5400) Option Price: $245.80
Actual Sell Put (K=$5200) Option Price: $162.20


#### e) Save the quote data to csv so that the following work can be recreated.

In [428]:
# Add a side column for clarity
buy_call.loc[:,'side'] = 'buy'
sell_call.loc[:,'side'] = 'sell'
buy_put.loc[:,'side'] = 'buy'
sell_put.loc[:,'side'] = 'sell'

# Combine the data
quote_data = pd.concat([buy_call, sell_call, buy_put, sell_put], ignore_index=True)

# Reorder columns for clarity
quote_data = quote_data[['side', 'optionType', 'strike', 'lastPrice', 'bid', 'ask', 'openInterest', 'volume', 'impliedVolatility']]

# Save quote data
quote_data.to_csv('quote_data.csv', index=False)

In [430]:
# Load quote data
quote_data = pd.read_csv('quote_data.csv')
quote_data

,side,optionType,strike,lastPrice,bid,ask,openInterest,volume,impliedVolatility
0,buy,call,5200.0,269.10,287.3,288.4,267,109.0,0.275465
1,sell,call,5400.0,151.53,169.4,171.3,39550,410.0,0.250551
2,buy,put,5400.0,249.10,244.0,245.8,54062,176.0,0.222948
3,sell,put,5200.0,164.18,162.2,163.7,35223,693.0,0.248323


#### f) Report the payoff at expiration, the profit, and the return.

The S&P 500 (^SPX) options multiplier is 100, meaning one option contract represents the potential to buy or sell 100 shares of the underlying index. When calculating the notional value (total amount) of an SPX option contract, we must multiply the option premium by 100 to reflect actual dollar amounts as experienced in trading.

In [434]:
# Calculate payoff at expiration
multiplier = 100  # ^SPX options multiplier
payoff_per_contract = (K2 - K1)  # Payoff is always K2 - K1
payoff = payoff_per_contract * multiplier

# Report results
print(f"Payoff at Expiration (1 Contract): ${payoff:.2f}")

Payoff at Expiration (1 Contract): $20000.00


In [436]:
# Initial cost from real quote data
total_outflow = actual_buy_call_price + actual_buy_put_price  # Options bought (ask price)
total_inflow = actual_sell_call_price + actual_sell_put_price  # Options sold (bid price)
initial_cost_per_contract = total_outflow - total_inflow
initial_cost = initial_cost_per_contract * multiplier

# Calculate profit
profit = payoff - initial_cost

# Report results
print(f"Initial Cost (1 Contract): ${initial_cost:.2f}")
print(f"Profit (1 Contract): ${profit:.2f}")

Initial Cost (1 Contract): $20260.00
Profit (1 Contract): $-260.00


In [438]:
# Calculate return
return_percentage = (profit / initial_cost) * 100

# Report results
print(f"Return: {return_percentage:.2f}%")

Return: -1.28%


The box spread results in a small loss due to the initial cost exceeding the payoff, likely due to transaction costs (bid-ask spreads)

#### g) Determine the “Greeks” for your strategy.

In [527]:
# BSM Greeks functions
def bsm_greeks(option_type, S0, K, T, r, sigma):
    # Get Parameters
    d1, d2 = bsm_parameters(S0, K, T, r, sigma)
    
    # Delta
    if option_type == 'call':
        delta = norm.cdf(d1)
    else:  # put
        delta = norm.cdf(d1) - 1
    
    # Gamma
    gamma = norm.pdf(d1) / (S0 * sigma * np.sqrt(T))
    
    # Theta
    if option_type == 'call':
        theta = -(S0 * sigma * norm.pdf(d1)) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    else:  # put
        theta = -(S0 * sigma * norm.pdf(d1)) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)
    
    # Vega
    vega = S0 * norm.pdf(d1) * np.sqrt(T)
    
    # Rho
    if option_type == 'call':
        rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    else:  # put
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2)
    
    return delta, gamma, theta, vega, rho

In [529]:
# Calculate Greeks for each option and sum for the strategy
total_delta = 0
total_gamma = 0
total_theta = 0
total_vega = 0
total_rho = 0

for _, option in quote_data.iterrows():
    delta, gamma, theta, vega, rho = bsm_greeks(option['optionType'], S0, option['strike'], T, r, option['impliedVolatility'])
    theta = theta / 252 # daily theta
    
    # Long positions add to the greeks, short positions subtract
    position = 1 if option['side'] == 'buy' else -1

    # Print option information
    print(f"{option['side']} {option['optionType']} (K={option['strike']}):")
    print(f"  Delta: {delta:.4f}, Gamma: {gamma:.4f}, Theta: {theta:.4f}, Vega: {vega:.4f}, Rho: {rho:.4f}")
    print(f"  Position: {position}\n")

    # Update greeks
    total_delta += position * delta
    total_gamma += position * gamma
    total_theta += position * theta
    total_vega += position * vega
    total_rho += position * rho

# Adjust for multiplier
total_delta *= multiplier
total_gamma *= multiplier
total_theta *= multiplier
total_vega *= multiplier
total_rho *= multiplier

# Report the Greeks
print("Greeks for the Box Spread Strategy (1 Contract):")
print(f"Delta: {total_delta:.4f}")
print(f"Gamma: {total_gamma:.4f}")
print(f"Theta: {total_theta:.4f} (per day)")
print(f"Vega: {total_vega:.4f}")
print(f"Rho: {total_rho:.4f}")

buy call (K=5200.0):
  Delta: 0.6052, Gamma: 0.0006, Theta: -3.1775, Vega: 840.9024, Rho: 494.0167
  Position: 1

sell call (K=5400.0):
  Delta: 0.4672, Gamma: 0.0007, Theta: -2.9122, Vega: 868.4168, Rho: 390.0311
  Position: -1

buy put (K=5400.0):
  Delta: -0.5416, Gamma: 0.0008, Theta: -1.7286, Vega: 866.6124, Rho: -528.6969
  Position: 1

sell put (K=5200.0):
  Delta: -0.3882, Gamma: 0.0007, Theta: -2.0471, Vega: 836.8987, Rho: -377.0380
  Position: -1

Greeks for the Box Spread Strategy (1 Contract):
Delta: -1.5507
Gamma: 0.0022
Theta: 5.3170 (per day)
Vega: 219.9337
Rho: -4767.3265


#### h) Comment on what you have found, is this value what you would expect? Will this strategy provide this return risk free? How will it react to different market changes? What would a trader need to take into account?

**Summarize Findings**

- Payoff at Expiration: 20,000 per contract (*K2 - K1* = 5400 − 5200 = 200 × 100).
- Profit: −260 (Payoff - Initial Cost = 20,000 − 20,260).
- Return: −1.28% (Profit / Initial Cost = −260/20,260 × 100).
- Greeks
  - Delta: -1.5507
  - Gamma: 0.0022
  - Theta: 5.3170 (per day)
  - Vega: 219.9337
  - Rho: -4767.3265


**Is This Value What We Would Expect?**

The strategy's payoff is exactly as expected. A box spread’s payoff at expiration is always *K2-K1*​ (200), times the multiplier 100 = 20,000, regardless of the underlying price at expiration. Contrarily, the initial cost of setting up the strategy, based on real quote data, deviates from 
the expected theorical cost. In a vaccum, with no transaction costs or market frictions, the initial cost should equal the present value of the payoff, i.e. *(K2-K1) x e^-rT*. The actual initial cost is higher than expected due to the bid-ask spread. As a consequence, the expected risk-free profit becomes negative profit because the actual cost exceeds the theoretical cost. The difference represents transaction costs (bid-ask spreads), leading to a loss. The negative return aligns with the negative profit. In a frictionless market, the return should be the risk-free rate (adjusted for the cost), but transaction costs cause a loss.

As for the greeks, a theoretically perfect box spread should yield a Delta, Gamma, Theta and Vega all close to 0, and a non-zero but small Rho, as explained above. However, the greeks deviate from the expected values given each option has a different implied volatility. In a perfect theoretical box spread, all volatilities would be the same, leading to an exact cancellation. Here, the differences prevent perfect cancellation.


**Will This Strategy Provide This Return Risk-Free?**

A box spread is designed to be a risk-free strategy because the payoff at expiration (𝐾2−𝐾1) is fixed, regardless of the underlying price. The return should theoretically equal the risk-free rate if the initial cost equals the present value of the payoff. However, as explored above, the strategy does not provide a risk-free return here, it in fact yields a negative return. In practice, transaction costs (bid-ask spreads, or even commissions, though not explored here) introduce a loss, making this strategy not truly risk-free. While the payoff is risk-free in terms of market risk (it’s fixed), the return is not risk-free due to transaction costs. In a frictionless market, the return would be risk-free and equal to the risk-free rate, but real-world frictions cause a loss.

**How Will The Strategy React to Different Market Changes?**

The Greeks provide insight into the strategy’s sensitivity to market changes.

Underlying Price Changes:
- Delta (-1.5507): A small negative Delta means the strategy’s value decreases slightly as the ^SPX price increases. However, this is minimal and should be ~0 in theory.
- Gamma (0.0022): A tiny positive Gamma indicates Delta becomes slightly less negative as the price moves, but this effect is negligible.
  
Reaction: The strategy is largely insensitive to price changes, as expected for a box spread.

Time Passage:
- Theta (5.3170 per day): The positive Theta means the strategy’s value increases slightly as time passes, which makes sense because the present value of the fixed payoff increases as expiration approaches (we’re closer to receiving 20,000).

Reaction: Time passage works in favor of the strategy, but the effect is small (5.32 per day).

Volatility Changes:
- Vega (219.9337): A positive Vega means the strategy’s value increases if implied volatility rises. This is unexpected for a box spread, where Vega should be ~0.

Reaction: The non-zero Vega is due to differing implied volatilities. If volatility increases uniformly, the strategy’s value may increase slightly, but this is a market inefficiency, not a feature of a perfect box spread.

Interest Rate Changes:
- Rho (-4767.3265): A negative Rho means the strategy’s value decreases if interest rates rise. Theoretically, Rho reflects the sensitivity of the present value of the payoff to interest rates.

Reaction: If interest rates increase by 1%, the value decreases by approximately $4,767, which is a significant sensitivity. This aligns with the nature of a box spread as an arbitrage strategy tied to the risk-free rate.


**What Would a Trader Need to Take into Account?**

A trader implementing this box spread strategy should consider:

- Transaction Costs: The bid-ask spread caused a loss, leading to a negative return. Traders must minimize these costs by negotiating better prices or trading in more liquid markets. Commissions and fees, even though not included here, would further reduce the return.

- Implied Volatility Mismatch: The differing implied volatilities led to non-zero Delta, Gamma, and Vega, introducing small market risks that shouldn’t exist in a theoretical box spread. A trader should try to execute the legs at similar implied volatilities to minimize these risks.

- Interest Rate Risk: The large negative Rho indicates significant sensitivity to interest rate changes. If rates rise, the strategy’s value decreases, which could exacerbate losses. Traders should understand and monitor interest rate expectations.

- Margin Requirements: Even though the payoff is fixed, brokers may require margin for the short positions. A trader needs to ensure they have sufficient capital and understand the margin impact.

- Arbitrage Opportunities: Theoretically, a box spread should yield the risk-free rate if priced correctly. Here, the negative return indicates no arbitrage opportunity. A trader should compare the cost to the risk-free rate and only execute if there’s a profit opportunity.

### 2. Expand your analysis to try to find the range of returns possible to achieve in the market with a box spread.

#### a) You are going to need many option prices for this part of the project. Please save these all to csv (and load them in) in such a way as that your results could be re-creatable.

In [773]:
def save_options_data(underlying, expiration, filename):
    ticker = yf.Ticker(underlying)
    options = ticker.option_chain(expiration)
    calls = options.calls
    puts = options.puts
    calls['optionType'] = 'call'
    puts['optionType'] = 'put'
    combined = pd.concat([calls, puts], ignore_index=True)
    combined.to_csv(filename, index=False)
    return combined

def load_options_data(filename):
    """
    Load options data from a CSV file.
    """
    return pd.read_csv(filename)

# Choosing 4 underlyings
underlyings = ['^SPX', '^NDX', 'AAPL', 'SPY']
options_data = {}

for underlying in underlyings:
    ticker = yf.Ticker(underlying)
    options_data[underlying] = {}

    # Select expirations that are approximately 1 month, 3 months, 6 months, and 12 months out.
    target_days = [30, 90, 180, 365]  # Target timeframes in days
    selected_expirations = []
    
    for target in target_days:
        target_date = today + datetime.timedelta(days=target)
        # Find the closest expiration date to the target date
        closest_exp = min(ticker.options, key=lambda x: abs((datetime.datetime.strptime(x, '%Y-%m-%d') - target_date).days))
        if closest_exp not in selected_expirations:
            selected_expirations.append(closest_exp)

    for i, expiration in enumerate(selected_expirations):
        # Define filename
        filename = f"{underlying.replace('^', '')}_{expiration}_options.csv"
        
        # Save options data to csv
        #save_options_data(underlying, expiration, filename)
        
        # Load options data
        options_data[underlying][f"{['1m', '3m', '6m', '1y'][i]}"] = load_options_data(filename)

In [775]:
# Load SPX options
SPX_1m = options_data['^SPX']['1m']
SPX_3m = options_data['^SPX']['3m']
SPX_6m = options_data['^SPX']['6m']
SPX_1y = options_data['^SPX']['1y']

# Load SPY options
NDX_1m = options_data['^NDX']['1m']
NDX_3m = options_data['^NDX']['3m']
NDX_6m = options_data['^NDX']['6m']
NDX_1y = options_data['^NDX']['1y']

# Load AAPL options
AAPL_1m = options_data['AAPL']['1m']
AAPL_3m = options_data['AAPL']['3m']
AAPL_6m = options_data['AAPL']['6m']
AAPL_1y = options_data['AAPL']['1y']

# Load SPY options
SPY_1m = options_data['SPY']['1m']
SPY_3m = options_data['SPY']['3m']
SPY_6m = options_data['SPY']['6m']
SPY_1y = options_data['SPY']['1y']

#### b) Report what you find in terms of possible returns.

In [778]:
def calculate_box_spread_return(quote_data, K1, K2, T, multiplier=100):
    """
    Calculate the annualized return for a box spread with strikes K1 and K2.
    """
    # Filter options for K1 and K2
    buy_call = quote_data[(quote_data['optionType'] == 'call') & (quote_data['strike'] == K1)]
    sell_call = quote_data[(quote_data['optionType'] == 'call') & (quote_data['strike'] == K2)]
    buy_put = quote_data[(quote_data['optionType'] == 'put') & (quote_data['strike'] == K2)]
    sell_put = quote_data[(quote_data['optionType'] == 'put') & (quote_data['strike'] == K1)]
    
    if buy_call.empty or sell_call.empty or buy_put.empty or sell_put.empty:
        return None
    
    # Use ask for buying, bid for selling
    initial_cost = (buy_call['ask'].values[0] + buy_put['ask'].values[0]) - \
                   (sell_call['bid'].values[0] + sell_put['bid'].values[0])
    initial_cost *= multiplier
    payoff = (K2 - K1) * multiplier
    if initial_cost <= 0:  # Avoid division by zero or negative cost
        return None
    annualized_return = (payoff / initial_cost) ** (1 / T) - 1
    return annualized_return

In [820]:
def select_strike_pairs(strikes, current_price, underlying):
    """
    Select multiple strike pairs for box spreads with varying widths.
    """
    strikes = sorted(strikes)
    strikes_below = [s for s in strikes if s < current_price]
    strikes_above = [s for s in strikes if s > current_price]
    
    if not strikes_below or not strikes_above:
        return []
    
    K1_base = np.max(strikes_below)  # Closest strike below current price
    K2_base = np.min(strikes_above)  # Closest strike above current price
    
    # Define base delta and multipliers based on underlying
    if underlying in ['^SPX', '^NDX']:
        base_delta = 100
        multipliers = [0, 1, 2, 3, 4]  # 0 for nearest pair, then wider spreads
    else:
        base_delta = 5
        multipliers = [0, 1, 2, 3, 4]
    
    strike_pairs = []
    for m in multipliers:
        K1 = K1_base - m * base_delta if K1_base - m * base_delta in strikes else K1_base
        K2 = K2_base + m * base_delta if K2_base + m * base_delta in strikes else K2_base
        if K1 < K2:  # Ensure K1 < K2
            strike_pairs.append((K1, K2))
    
    return strike_pairs

In [840]:
underlyings = ['^SPX', '^NDX', 'AAPL', 'SPY']
risk_free_rate = get_risk_free_rate()
options = {}
returns = []

for underlying in underlyings:
    ticker = yf.Ticker(underlying)
    current_price = ticker.history(start=yesterday, end=today)['Close'].iloc[-1]
    print(f'{underlying} Current Price: {current_price:.2f}')

    # Select expirations that are approximately 1 month, 3 months, 6 months, and 12 months out.
    target_days = [30, 90, 180, 365]  # Target timeframes in days
    selected_expirations = []
    
    for target in target_days:
        target_date = today + datetime.timedelta(days=target)
        # Find the closest expiration date to the target date
        closest_exp = min(ticker.options, key=lambda x: abs((datetime.datetime.strptime(x, '%Y-%m-%d') - target_date).days))
        if closest_exp not in selected_expirations:
            selected_expirations.append(closest_exp)
            
    for expiration in selected_expirations:
        # Define filename
        filename = f"{underlying.replace('^', '')}_{expiration}_options.csv"
        
        # Load options data
        options[underlying] = load_options_data(filename)
        
        # Select strikes: K1 slightly below the current price and K2 slightly above
        strikes = options[underlying]['strike'].unique()

        # Generate strike pairs dynamically
        strike_pairs = select_strike_pairs(strikes, current_price, underlying)

        # Calculate time to expiration in years
        T = time_to_expiration(expiration)

        for K1, K2 in strike_pairs:
            if K2 in strikes:
                ret = calculate_box_spread_return(options[underlying], K1, K2, T)
                if ret is not None:
                    returns.append(ret)
                    print(f"Expiration: {expiration}, Strikes {K1}-{K2}: "
                            f"Annualized Return = {ret:.2%}")

    print('\n')

^SPX Current Price: 5287.76
Expiration: 2025-05-23, Strikes 5280.0-5290.0: Annualized Return = -100.00%
Expiration: 2025-05-23, Strikes 5180.0-5390.0: Annualized Return = -49.72%
Expiration: 2025-05-23, Strikes 5080.0-5490.0: Annualized Return = -24.59%
Expiration: 2025-05-23, Strikes 4980.0-5590.0: Annualized Return = -14.67%
Expiration: 2025-05-23, Strikes 4880.0-5290.0: Annualized Return = -31.49%
Expiration: 2025-07-18, Strikes 5285.0-5290.0: Annualized Return = -98.73%
Expiration: 2025-07-18, Strikes 5185.0-5390.0: Annualized Return = -1.55%
Expiration: 2025-07-18, Strikes 5085.0-5490.0: Annualized Return = -0.88%
Expiration: 2025-07-18, Strikes 4985.0-5590.0: Annualized Return = -2.92%
Expiration: 2025-07-18, Strikes 5285.0-5690.0: Annualized Return = -3.76%
Expiration: 2025-10-17, Strikes 5280.0-5290.0: Annualized Return = -69.57%
Expiration: 2025-10-17, Strikes 5180.0-5390.0: Annualized Return = -0.37%
Expiration: 2026-04-17, Strikes 5275.0-5300.0: Annualized Return = -27.81%
E

In [844]:
# Analyze results
best_return = np.max(returns)
worst_return = np.min(returns)
print("\n### Results ###")
print(f"Best Annualized Return: {best_return:.2%}")
print(f"Worst Annualized Return: {worst_return:.2%}")
print(f"Range of Returns: {worst_return:.2%} to {best_return:.2%}")
print(f"Risk-Free Rate Reference: {risk_free_rate:.2%}")


### Results ###
Best Annualized Return: 2.22%
Worst Annualized Return: -100.00%
Range of Returns: -100.00% to 2.22%
Risk-Free Rate Reference: 4.21%


#### c) Discuss what you found, were there any differences? Why? How did you determine best or worst?

In analyzing box spreads across various underlyings (^SPX, ^NDX, AAPL, SPY), expiration dates, and strike pairs, I found a wide range of annualized returns, spanning from -100.00% to 2.22%. A box spread, theoretically a risk-free options strategy with a fixed payoff at expiration, should ideally yield returns close to the risk-free rate (given as 4.21%). However, the observed returns indicate significant deviations, with the best return of 2.22% still falling short of the risk-free rate and the worst return of -100.00% suggesting substantial losses in some cases.

There were notable differences in the annualized returns across the underlyings, expiration dates, and strike pairs. These differences manifest in several ways:

**Across Underlyings**
- ^SPX (S&P 500 Index): Returns ranged from -100.00% to 2.22%, showing the widest spread and including the highest positive return observed.
- ^NDX (NASDAQ-100 Index): Returns were consistently negative, ranging from -100.00% to -9.46%, with no positive returns.
- AAPL (Apple Stock): Returns ranged from -37.61% to 0.95%, showing a mix of negative and slightly positive returns.
- SPY (S&P 500 ETF): Returns ranged from -97.29% to 1.85%, similar to ^SPX but with a slightly lower maximum return.

**Across Expiration Dates**
- Shorter-dated expirations (1 Month) frequently exhibited the worst returns.
- Longer-dated expirations tended to produce better returns, including the highest values like 2.22% and 0.95%, and the least negative value for ^NDX, -9.47%.

**Across Strike Spread Widths**
- Narrow strike spreads (e.g., 5280.0-5290.0 for ^SPX, a 10-point spread) often resulted in highly negative returns.
- Wider strike spreads (e.g., 4975.0-5600.0 for ^SPX, a 625-point spread or 175.0-220.0 for AAPL, a 45-point spread) were associated with the better returns, such as 2.22% and 0.95%, respectively.


Several factors contributed to the observed differences in returns:

**Transaction Costs (Bid-Ask Spreads):**
Box spreads involve four options contracts, amplifying the impact of bid-ask spreads. For narrow spreads and short-dated options, the initial cost often exceeded the payoff due to wide spreads relative to the small potential gain, leading to returns like -100.00%. Wider spreads and longer expirations diluted the relative impact of these costs, improving returns.

**Liquidity of Options:**
Underlyings like ^SPX and SPY, tied to broad market indices, have more liquid options markets, resulting in tighter spreads and occasionally positive returns. Conversely, ^NDX showed consistently negative returns, possibly due to lower liquidity.

**Time to Expiration:**
Shorter expirations amplify the effect of transaction costs over a smaller time frame, leading to highly negative annualized returns. Longer expirations spread these costs over more time, reducing their annualized impact and allowing returns to approach or exceed zero.

**Strike Spread Width:**
Wider spreads increase the payoff (difference between strikes), which can offset transaction costs more effectively. Narrow spreads, with smaller payoffs, are more sensitive to costs, often resulting in negative returns.

**Market Frictions:**
In a frictionless market, box spreads should yield the risk-free rate (4.21%). However, real-world frictions like transaction costs and imperfect pricing prevent this, causing returns to vary widely and rarely reach the theoretical ideal.

**How Did I Determine the Best and Worst Returns?**

To identify the best and worst returns, I examined the annualized returns calculated for each combination of underlying, expiration date, and strike pair from the provided data.
The best return was determined as the maximum value: 2.22% (^SPX, expiration 2026-04-17, strikes 4975.0-5600.0).
The worst return was the minimum value: -100.00%, observed in multiple instances (^SPX, expiration 2025-05-23, strikes 5280.0-5290.0 and ^NDX, expiration 2025-05-23, strikes 18250.0-18300.0).


**Conclusion**

The box spread analysis revealed that returns deviate significantly from the theoretical risk-free rate due to market frictions, even in assets with liquid options. Differences across underlyings, expirations, and strike widths highlight the critical roles of transaction costs, liquidity, and time to expiration. The best returns, while positive, fall below the risk-free rate, and the worst indicate potential large losses, particularly in less favorable conditions. These insights were derived by systematically comparing all calculated returns to pinpoint the extremes.